In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
import os

TICKER = "AAPL"
START = "2010-01-01"
END = "2025-01-01"

df = yf.download(TICKER, start=START, end=END)

df = df.droplevel('Ticker', axis=1)
df.columns.name = None

df.to_csv(f"../data/raw/{TICKER}.csv")

In [48]:
df=df.droplevel('Ticker', axis=1)
df.columns.name = None

In [49]:
df

,Close,High,Low,Open,Volume
Date,,,,,
2010-01-04,6.418384,6.433079,6.369498,6.400989,493729600
2010-01-05,6.429480,6.465769,6.395589,6.436078,601904800
2010-01-06,6.327211,6.454973,6.320613,6.429481,552160000
2010-01-07,6.315513,6.358101,6.269627,6.350603,477131200
2010-01-08,6.357500,6.358100,6.269927,6.307115,447610800
...,...,...,...,...,...
2024-12-24,257.037506,257.047440,254.140589,254.339701,23234700
2024-12-26,257.853760,258.928914,256.470034,257.027510,27237100
2024-12-27,254.439224,257.535238,251.920617,256.669129,42355300


In [22]:
SEQ_LEN = 30

df = pd.read_csv(f"../data/raw/{TICKER}.csv", parse_dates=True)
# df["Return"] = df["Close"].pct_change().shift(-1)
# df.dropna(inplace=True)

# df["Label"] = np.where(df["Return"]>0.001, 2, np.where(df["Return"]<-0.001, 0, 1))

# X, y = [], []
# for i in range(df.shape[0]-SEQ_LEN):
#     X.append(df[["Open","High","Low","Close","Volume"]].iloc[i:i+SEQ_LEN].values)
#     y.append(df["Label"].iloc[i+SEQ_LEN])

# X = np.array(X)
# y = np.array(y)
# print(X.shape, y.shape)
df

,Price,Close,High,Low,Open,Volume
0,Ticker,AAPL,AAPL,AAPL,AAPL,AAPL
1,Date,NaN,NaN,NaN,NaN,NaN
2,2010-01-04,6.418383598327637,6.433079146557626,6.369498276526275,6.400988794222488,493729600
3,2010-01-05,6.429479598999023,6.465768788500255,6.3955893472790475,6.436077779062374,601904800
4,2010-01-06,6.327211380004883,6.454973360062211,6.320613198722003,6.429480787347484,552160000
...,...,...,...,...,...,...
3771,2024-12-24,257.0375061035156,257.04744042152294,254.14058909817473,254.3397007808045,23234700
3772,2024-12-26,257.853759765625,258.9289141129315,256.47003399402934,257.0275101814573,27237100
3773,2024-12-27,254.43922424316406,257.53523753189216,251.92061656694878,256.66912911312295,42355300
3774,2024-12-30,251.0644989013672,252.3586488565982,249.62103037787767,251.0943626150189,35557500


In [15]:
df.shape[0], len(df)

(3774, 3774)